In [1]:
import sqlite3
import pandas as pd

In [2]:
DATABASE_PATH = "./data/air-quality-data/fortaleza_trama_data_hour.sqlite"

In [6]:
def explore_database(db_path):
    # Conectar ao banco de dados SQLite
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # Listar todas as tabelas no banco de dados
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tabelas no banco de dados:")
    for table in tables:
        print(f"- {table[0]}")

    # Explorar estrutura da tabela principal
    if tables:
        main_table = tables[0][0]  # Seleciona a primeira tabela encontrada
        print(f"\nEstrutura da tabela '{main_table}':")
        cursor.execute(f"PRAGMA table_info({main_table});")
        columns = cursor.fetchall()
        for col in columns:
            print(f"- {col[1]} (tipo: {col[2]})")

        # Carregar os primeiros 10 dados da tabela principal
        print(f"\nPrimeiros 10 registros da tabela '{main_table}':")
        df = pd.read_sql_query(f"SELECT * FROM {main_table} LIMIT 10;", conn)
        print(df)
    else:
        print("Nenhuma tabela encontrada no banco de dados.")

    # Fechar a conexão
    conn.close()

# Chamada da função
explore_database(DATABASE_PATH)

Tabelas no banco de dados:
- trama_moqabase_hour
- metadata
- trama_moqabase_hour_normalized

Estrutura da tabela 'trama_moqabase_hour':
- date (tipo: TEXT)
- time (tipo: TEXT)
- moqa_id (tipo: TEXT)
- internal_temp (tipo: REAL)
- external_temp (tipo: REAL)
- humidity (tipo: REAL)
- pressure (tipo: REAL)
- pm1 (tipo: REAL)
- pm25 (tipo: REAL)
- pm10 (tipo: REAL)
- pms_log (tipo: REAL)
- bme_log (tipo: REAL)
- latitude (tipo: REAL)
- longitude (tipo: REAL)

Primeiros 10 registros da tabela 'trama_moqabase_hour':
         date      time moqa_id  internal_temp  external_temp   humidity  \
0  2024-08-01  07:00:00  A5BBFC      29.275969      25.457152  55.517520   
1  2024-08-01  08:00:00  A5BBFC      30.967749      27.273467  48.863613   
2  2024-08-01  09:00:00  A5BBFC      31.919670      28.814042  45.266362   
3  2024-08-01  10:00:00  A5BBFC      33.363878      29.678385  43.889363   
4  2024-08-01  11:00:00  A5BBFC      34.342411      29.788842  41.941384   
5  2024-08-01  12:00:00  A5

In [3]:
def delete_unwanted_ids(db_path, unwanted_ids):
    """
    Deleta as linhas do banco de dados onde moqa_id corresponde a um dos IDs indesejados.
    
    Args:
        db_path (str): Caminho para o banco de dados SQLite.
        unwanted_ids (list): Lista de IDs indesejados.
    """
    try:
        # Conectar ao banco de dados
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Deletar linhas com moqa_id indesejado
        placeholders = ", ".join("?" for _ in unwanted_ids)  # Para usar no SQL IN
        query = f"DELETE FROM trama_moqabase_hour_normalized WHERE moqa_id IN ({placeholders});"
        cursor.execute(query, unwanted_ids)
        
        # Confirmar alterações
        conn.commit()
        print(f"{cursor.rowcount} linhas deletadas com os IDs indesejados.")
        
    except sqlite3.Error as e:
        print(f"Erro ao acessar o banco de dados: {e}")
    finally:
        # Fechar conexão
        conn.close()

# Lista de IDs indesejados
unwanted_ids = ["JJJAAA", "JJJBBB", "JJJCCC", "JJJDDD", "JJJEEE", "NOVAT"]

# Chamar a função
delete_unwanted_ids(DATABASE_PATH, unwanted_ids)

36 linhas deletadas com os IDs indesejados.


In [7]:
def get_unique_values(db_path, table_name, column_name):
    """
    Obtém todos os valores únicos de uma coluna específica em uma tabela do banco de dados.
    
    Args:
        db_path (str): Caminho para o banco de dados SQLite.
        table_name (str): Nome da tabela.
        column_name (str): Nome da coluna.
    
    Returns:
        list: Lista de valores únicos encontrados.
    """
    try:
        # Conectar ao banco de dados
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        
        # Consultar valores únicos
        query = f"SELECT DISTINCT {column_name} FROM {table_name};"
        cursor.execute(query)
        unique_values = [row[0] for row in cursor.fetchall()]
        
        print(f"Valores únicos na coluna '{column_name}':")
        for value in unique_values:
            print(value)
        
        return unique_values
        
    except sqlite3.Error as e:
        print(f"Erro ao acessar o banco de dados: {e}")
        return []
    finally:
        # Fechar conexão
        conn.close()

# Chamar a função para obter os valores únicos de moqa_id
unique_moqa_ids = get_unique_values(DATABASE_PATH, "trama_moqabase_hour_normalized", "moqa_id")

Valores únicos na coluna 'moqa_id':
A5BBFC
AV1CD2
AV4STA
AV5CEC
DC40AC
DC663C
DCB604
DD31EC
DD32D4
DD3468
DD34F0
DD56F0
DDCA1C
JJJAAA
JJJBBB
JJJCCC
JJJDDD
JJJEEE
NOVAT
